### GDP Energy

In [1]:
import pandas as pd
import plotly.express as px
from graphs import apply_template

BEGIN_YEAR = 1965

ModuleNotFoundError: No module named 'graphs'

In [ ]:
gdp = pd.read_csv("data/gdp.csv", skiprows=3).drop(
    ["Country Code", "Indicator Code", "Indicator Name"], axis=1
)
gdp = (
    gdp.loc[gdp["Country Name"] == "World"]
    .T.drop(["Country Name", "Unnamed: 68"])[259]
    .rename("gdp")
    .astype(float)
)
gdp.index = gdp.index.astype(int)
gdp = gdp.loc[BEGIN_YEAR:]

In [ ]:
energy = pd.read_csv("data/energy.csv").set_index("Year")

In [ ]:
df = pd.concat([energy, gdp], axis=1)
df.index.name = "year"

In [ ]:
co2 = pd.read_csv("data/co2-emissions-and-gdp.csv")
print(co2.columns)
co2 = co2.loc[
    co2["Entity"] == "World",
    ["Year", "GDP, PPP (constant 2017 international $)", "Annual CO₂ emissions"],
].set_index("Year")

In [ ]:
df["co2"] = co2["Annual CO₂ emissions"]

In [ ]:
df = df.reset_index()
df["year"] = df["year"].astype(int)
df = df.loc[df["year"] >= 1980]

In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy import stats

# Assuming df is your existing dataframe
# Make sure df is sorted by year
df = df.sort_values("year")

x_column = "gdp"
y_column = "co2"
# Create the scatter plot

fig = go.Figure(
    go.Scatter(
        x=df[x_column],
        y=df[y_column],
        mode="lines+markers",  # This tells Plotly to draw both lines and markers
        line=dict(color="black", width=2),
        marker=dict(
            color="black", size=5, symbol="circle", line=dict(color="black", width=2)
        ),
        showlegend=False,
    )
)

# Add regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(
    df[x_column], df[y_column]
)
line_x = np.array([df[x_column].min(), df[x_column].max()])
line_y = slope * line_x + intercept
print(line_x, line_y)
fig.add_trace(
    go.Scatter(
        x=line_x,
        y=line_y,
        name="Ligne X = Y",
        line=dict(color="black", dash="dash"),
        showlegend=True,
        mode="lines",
    )
)

apply_template(fig)
# years_to_annotate = list(range(1990, 2021, 10)) + [2023]
# for year in years_to_annotate:
#     year_data = df[df["year"] == year]
#     if not year_data.empty:
#         fig.add_annotation(
#             x=year_data[x_column].iloc[0],
#             y=year_data[y_column].iloc[0],
#             text=f"{year:.0f}",
#             showarrow=False,
#             yshift=10,
#             font=dict(size=12),
#         )

# Update layout
fig.update_layout(width=650, height=600)
fig.update_xaxes(title="GDP in current USD")
fig.update_yaxes(title="CO2e emissions")
# Show the plot
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy import stats

gdp_name = "GDP, PPP (constant 2017 international $)"
co2_name = "Annual CO₂ emissions"

# Assuming co2 is your existing dataframe
# Make sure df is sorted by year
co2 = co2.reset_index().set_index("Year").sort_index()

# Create the scatter plot

fig = go.Figure(
    go.Scatter(
        x=co2[gdp_name],
        y=co2[co2_name],
        mode="lines+markers",  # This tells Plotly to draw both lines and markers
        line=dict(color="black", width=2),
        marker=dict(
            color="black", size=5, symbol="circle", line=dict(color="black", width=2)
        ),
        showlegend=False,
    )
)


# Add regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(
    co2[gdp_name], co2[co2_name]
)
line_x = np.array([co2[gdp_name].min(), co2[gdp_name].max()])
line_y = slope * line_x + intercept

fig.add_trace(
    go.Scatter(
        x=line_x,
        y=line_y,
        name="Ligne X = Y",
        line=dict(color="black", dash="dash"),
        showlegend=False,
        mode="lines+markers",
    )
)

# Update layout
apply_template(fig)
years_to_annotate = list(range(1990, 2021, 10))
for year in years_to_annotate:
    year_data = co2.loc[year]
    fig.add_annotation(
        x=year_data[gdp_name],
        y=year_data[co2_name],
        text=f"{year:.0f}",
        yshift=30,
        showarrow=False,
        font=dict(size=14, color="grey", weight="bold"),
    )
fig.update_layout(width=700, height=600)
fig.update_xaxes(title="PIB en dollars U.S. 2017")
fig.update_yaxes(title="Emissions de CO2e")

# Show the plot
fig.show()